In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mnist_utils
import random
import relu_utils as alg
import spiking_relu as sr
import copy
import scipy
import pickle

In [2]:
cell_params_lif = {'cm': 0.25,      #nF
                   'i_offset': 0.1, #nA
                   'tau_m': 20.0,   #ms
                   'tau_refrac': 1.,#ms
                   'tau_syn_E': 5.0,#ms
                   'tau_syn_I': 5.0,#ms
                   'v_reset': -65.0,#mV
                   'v_rest': -65.0, #mV
                   'v_thresh': -50.0#mV
                   }

#Figure 2: noisy_current against firing rate

In [3]:
def noise_current(current, stdnoise, cell_params_lif, trial):
    import pyNN.nest as p
    runtime = 10000.
    p.setup(timestep=1.0, min_delay=1.0, max_delay=16.0)
    poplist = []
    ratelist = np.zeros(trial)
    for i in range(trial):
        pop = p.Population(1, p.IF_curr_exp, cell_params_lif)
        noise = p.NoisyCurrentSource(mean=current, stdev=stdnoise, start=0, stop=runtime, dt=1.0,rng=p.NativeRNG(seed=i))
        noise.inject_into(pop)
        pop.record()
        poplist.append(pop)
    p.run(runtime)
    for i in range(trial):
        spikes = poplist[i].getSpikes(compatible_output=True)
        ratelist[i] = len(spikes)*1000./runtime
    p.end()
    return ratelist.mean(), ratelist.max(), ratelist.min()

In [4]:
trial = 10
stdnoise = np.array([0.2, 0.5, 1.])
current =  np.arange(-0.5,0.65,0.1)
# current = np.append(current, np.arange(0.5,1.2,0.2))

x_num = current.size
noise_num = stdnoise.size
rate_mean = np.zeros((x_num, stdnoise.size))
rate_max = np.zeros((x_num, stdnoise.size))
rate_min = np.zeros((x_num, stdnoise.size))
for j in range(noise_num):
    for i in range(x_num):
        rate_mean[i,j], rate_max[i,j], rate_min[i,j] = noise_current(current[i], stdnoise[j], cell_params_lif,trial)
#plt.plot(current, rate_mean, 'green')
#plt.fill_between(current, rate_min, rate_max, facecolor='green', alpha=0.5, interpolate=True)

In [9]:
with open('rate1.pickle', 'w') as f:
    pickle.dump([rate_mean, rate_max, rate_min], f)

#Figure 3: noisy current generated by Poisson spikes vs. firing rate
y_noise = np.sqrt(signal.convolve2d(abs(xi),np.transpose(w**2.),mode='valid'))

In [3]:
def gen_spikesource(num, dur, rate):
    from mnist_utils import poisson_generator
    spike_source_data = [[] for i in range(num)]
    for i in range(num):
        t_start = 0
        t_stop = dur
        spikes = poisson_generator(rate, t_start, t_stop)
        if spikes != []:
            spike_source_data[i].extend(spikes)
    return spike_source_data

In [4]:
def noise_poisson(mu, sigma, trial):
    import pyNN.nest as p
    runtime = 10000.
    p.setup(timestep=1.0, min_delay=1.0, max_delay=16.0)
    
    num = 500
    w = 0.05
    
    rate_sub = mu/w
    rate_sum = 2.*sigma**2./w**2.
    
    rate1 = (rate_sub + rate_sum)/2.
    rate2 = (rate_sum - rate1)
    rate1 = rate1 * 1000./cell_params_lif['tau_syn_E']/float(num)
    rate2 = rate2 * 1000./cell_params_lif['tau_syn_I']/float(num)
    poplist = []
    ratelist = np.zeros(trial)
    
    np.random.seed(0)
    for i in range(trial):
        spike_source_data1 = gen_spikesource(num, runtime, rate1)
        spike_source_data2 = gen_spikesource(num, runtime, rate2)
        pop = p.Population(1, p.IF_curr_exp, cell_params_lif)
        noise1 = p.Population(num,  p.SpikeSourceArray, {'spike_times' : []})
        noise2 = p.Population(num,  p.SpikeSourceArray, {'spike_times' : []})
        for j in range(num):
            noise1[j].spike_times = spike_source_data1[j]
            noise2[j].spike_times = spike_source_data2[j]
            
        ee_connector = p.AllToAllConnector(weights=w)
        in_connector = p.AllToAllConnector(weights=-w)
        p.Projection(noise1, pop, ee_connector, target='excitatory')
        p.Projection(noise2, pop, in_connector, target='inhibitory')
        pop.record()
        poplist.append(pop)
    p.run(runtime)
    for i in range(trial):
        spikes = poplist[i].getSpikes(compatible_output=True)
        ratelist[i] = len(spikes)*1000./runtime
    p.end()
    return ratelist.mean(), ratelist.max(), ratelist.min()

In [12]:
trial = 10
stdnoise = np.array([0.2, 0.5, 1.])
current =  np.arange(-0.5,0.65,0.1)
x_num = current.size
noise_num = stdnoise.size
rate_mean2 = np.zeros((x_num, stdnoise.size))
rate_max2 = np.zeros((x_num, stdnoise.size))
rate_min2 = np.zeros((x_num, stdnoise.size))
for j in range(noise_num):
    for i in range(x_num):
        rate_mean2[i,j], rate_max2[i,j], rate_min2[i,j] = noise_poisson(current[i], stdnoise[j],trial)

In [14]:
with open('rate2.pickle', 'w') as f:
    pickle.dump([rate_mean2, rate_max2, rate_min2], f)

In [5]:
trial = 10
stdnoise = np.array([0.2, 0.5, 1.])
current =  np.arange(-0.5,0.65,0.1)
cell_params_lif = {'cm': 0.25,      #nF
                   'i_offset': 0.1, #nA
                   'tau_m': 20.0,   #ms
                   'tau_refrac': 1.,#ms
                   'tau_syn_E': 1.0,#ms
                   'tau_syn_I': 1.0,#ms
                   'v_reset': -65.0,#mV
                   'v_rest': -65.0, #mV
                   'v_thresh': -50.0#mV
                   }
x_num = current.size
noise_num = stdnoise.size
rate_mean3 = np.zeros((x_num, stdnoise.size))
rate_max3 = np.zeros((x_num, stdnoise.size))
rate_min3 = np.zeros((x_num, stdnoise.size))
for j in range(noise_num):
    for i in range(x_num):
        rate_mean3[i,j], rate_max3[i,j], rate_min3[i,j] = noise_poisson(current[i], stdnoise[j],trial)

In [6]:
with open('rate3.pickle', 'w') as f:
    pickle.dump([rate_mean3, rate_max3, rate_min3], f)